In [4]:
!pip install python-telegram-bot

     |████████████████████████████████| 450kB 7.5MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 


In [5]:
!pip install surprise

     |████████████████████████████████| 11.8MB 6.3MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617546 sha256=51b46c20380a6e915e7fe6ffe0d030b783215e46356ab0a01df8a846084fb6e6
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


# data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/My Drive/GitHub/minor-goodbooks-recommender

/content/drive/My Drive/GitHub/minor-goodbooks-recommender


In [3]:
import pandas as pd
import numpy as np
import time

from surprise import Reader, Dataset
from surprise import KNNWithMeans, SVD, SVDpp

In [4]:
ratings = pd.read_csv('data/ratings.csv')
ratings_random = pd.read_csv('data/ratings_random.csv')

In [5]:
books = pd.read_csv('data/books.csv')
book_map = books[['id', 'title']]

In [6]:
def recommend_list(user_ratings, ratings_data, algorithm, verbose = False):
  start = time.time()

  reader = Reader(rating_scale=(1, 5))
  #data = Dataset.load_from_df(ratings_data, reader)
  data_full = Dataset.load_from_df(ratings_data.append(user_ratings), reader).build_full_trainset()

  algorithm.fit(data_full)

  preds = []
  for i in range(1, 9901):
    preds.append(algorithm.predict(user_ratings.user_id.unique()[0], i).est)
  
  recs = pd.DataFrame({'book_id' : range(1,9901), 'estimated_rating' : preds})
  recs = recs.loc[~recs['book_id'].isin(user_ratings['book_id'])].sort_values('estimated_rating', ascending = False).head(10)
  if verbose:
    print('Seconds: ', time.time()-start)

  return recs

In [7]:
def fetch_user_ratings_id(user_id):
  user_ratings = ratings.loc[ratings['user_id']==user_id, :]
  return user_ratings

In [8]:
def fetch_user_ratings_goodreads(goodreads_id):
  import requests

  url = f"https://www.goodreads.com/review/list/{goodreads_id}?print=true"
  headers = {
      'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
      (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }

  page = requests.get(url=url, headers = headers).text

  for i in range(2, 11):
    url = f"https://www.goodreads.com/review/list/{goodreads_id}?page={i}&print=true"
    page += requests.get(url=url, headers = headers).text

  from bs4 import BeautifulSoup
  soup = BeautifulSoup(page, 'html.parser')

  bad_stuff_titles = ['Goodreads Home', None, 'My group discussions', 'Messages', 'Friends', 
                      'did not like it', 'it was ok', 'liked it', 'really liked it', 'it was amazing',]
  titles = [i.get('title') for i in soup.find_all('a')]
  books = [i for i in titles if i not in bad_stuff_titles]
  
  bad_stuff_ratings = [None,
                      ['headerPersonalNav__icon', 'headerPersonalNav__icon--notifications'],
                      ['headerPersonalNav__icon', 'headerPersonalNav__icon--discussions'],
                      ['headerPersonalNav__icon', 'headerPersonalNav__icon--inbox'],
                      ['headerPersonalNav__icon', 'headerPersonalNav__icon--friendRequests'],
                      ['controlGroup', 'uitext'],
                      ['bookMeta'],
                      ['greyText'],
                      ['greyText', 'smallText'],
                      ['greyText'],
                      ['', 'staticStars', 'notranslate'],
                      ['darkGreyText'], 
                      ['date_started_value'], 
                      ['date_read_value'],
                      ['previous_page', 'disabled'],
                      ['next_page', 'disabled'],]
  rates = [i.get('class') for i in soup.find_all('span')]
  stars = [i[1] for i in rates if i not in bad_stuff_ratings]

  def groupwise(iterable):
    a = iter(iterable)
    return zip(a, a, a, a, a)

  ratings = [(s1, s2, s3, s4, s5).count('p10') for (s1, s2, s3, s4, s5) in groupwise(stars)]

  df = pd.merge(pd.DataFrame({'title' : books, 'rating' : ratings}), book_map, on='title')
  df = df.drop('title', axis=1).rename({'id' : 'book_id'}, axis=1)
  df['user_id'] = [999999] * len(df)
  df = df.reindex(columns=['user_id', 'book_id', 'rating'])
  df = df.loc[df['rating'] != 0]

  if df.empty:
    raise ValueError('No matching books rated')
  else: return df

# bot

In [9]:
#import logging
#from typing import Tuple, Dict, Any

from telegram import Update, ReplyKeyboardMarkup
from telegram.ext import (
    Updater,
    CommandHandler,
    MessageHandler,
    Filters,
    ConversationHandler,
    CallbackQueryHandler,
    CallbackContext,
)

In [10]:
# State definitions for top level conversation
CHOOSING_SCENARIO, CHOOSING_USER_ID, CHOOSING_GR_ID, CHOOSING_CUSTOM = map(chr, range(4))
# State definitions for second level conversation
SAVING_USER_ID, SELECTING_GENDER = map(chr, range(4, 6))
# State definitions for descriptions conversation
SELECTING_ENGINE, TYPING_USER, TYPING_GOODREADS = map(chr, range(6, 9))
# Meta states
STOPPING, SHOWING = map(chr, range(9, 11))
# Shortcut for ConversationHandler.END
END = ConversationHandler.END

In [11]:
from telegram.ext import Updater
from getpass import getpass
updater = Updater(token=getpass(), use_context=True)

··········


In [65]:
#import logging
#logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
#                     level=logging.INFO)

In [12]:
def start(update, context):
    context.user_data['user_ratings'] = None
    text = "Choose scenario"

    buttons = [[
                'User Id',
                'GoodReads Id',
                #'Custom Setup'
                ]]
    keyboard = ReplyKeyboardMarkup(buttons, one_time_keyboard=True)
    update.message.reply_text(text=text, reply_markup=keyboard)

    return CHOOSING_SCENARIO

In [13]:
def recommend(update, context):
  text = "Choose recommender engine"

  buttons = [['KNN', 'SVD']]

  keyboard = ReplyKeyboardMarkup(buttons, one_time_keyboard=True)
  update.message.reply_text(text=text, reply_markup=keyboard)

  return SELECTING_ENGINE

In [14]:
def ask_user_id(update, context):
  text = 'Type a number between 0 and ' + str(ratings.user_id.nunique())
  update.message.reply_text(text=text)
  return TYPING_USER

def ask_user_id_again(update, context):
  text = 'Wrong input. Type a number between 0 and ' + str(ratings.user_id.nunique())
  update.message.reply_text(text=text)
  return TYPING_USER

def save_user_id(update, context):
  try:
    user_id = int(update.message.text)
    if user_id not in range(ratings.user_id.nunique()+1):
      raise ValueError
    context.user_data['user_ratings'] = fetch_user_ratings_id(user_id)
    return recommend(update, context)
  except ValueError:
    return ask_user_id_again(update, context)

In [15]:
def ask_goodreads_id(update, context):
  text = 'Type your GoodReads Id'
  update.message.reply_text(text=text)
  return TYPING_GOODREADS

def ask_goodreads_id_again(update, context):
  text = 'User does not exist or has no books from dataset rated. Please try again or choose different scenario by calling /start'
  update.message.reply_text(text=text)
  return TYPING_GOODREADS

def save_goodreads_id(update, context):
  try:
    user_id = int(update.message.text)
    context.user_data['user_ratings'] = fetch_user_ratings_goodreads(user_id)
    return recommend(update, context)
  except ValueError:
    return ask_goodreads_id_again(update, context)

In [16]:
def warn_knn(update, context):
  text = 'This takes time'
  update.message.reply_text(text=text)
  return rec_knn(update, context)

def rec_knn(update, context):
  knn = KNNWithMeans(k=9, verbose=False)
  rec_table = recommend_list(context.user_data['user_ratings'], ratings_random, knn, verbose = False)

  rec_table = pd.merge(rec_table, book_map.rename({'id' : 'book_id'}, axis = 1), on='book_id')

  text = ''
  for i, title in enumerate(rec_table['title']):
    text = text + str(i+1) + '. ' + str(title) + '\n'

  update.message.reply_text(text=text)

In [17]:
def warn_svd(update, context):
  text = 'This takes time'
  update.message.reply_text(text=text)
  return rec_svd(update, context)

def rec_svd(update, context):
  svd = SVDpp(n_factors=20, verbose=False)
  rec_table = recommend_list(context.user_data['user_ratings'], ratings_random, svd, verbose = False)

  rec_table = pd.merge(rec_table, book_map.rename({'id' : 'book_id'}, axis = 1), on='book_id')

  text = ''
  for i, title in enumerate(rec_table['title']):
    text = text + str(i+1) + '. ' + str(title) + '\n'

  update.message.reply_text(text=text)

In [18]:
dispatcher = updater.dispatcher

In [19]:
dispatcher.add_handler(ConversationHandler(
  entry_points = [CommandHandler('start', start)],
  states = {
      CHOOSING_SCENARIO: [MessageHandler(Filters.regex('^User Id$'), ask_user_id),
                          MessageHandler(Filters.regex('^GoodReads Id$'), ask_goodreads_id)],
      TYPING_USER: [MessageHandler(Filters.text & ~Filters.command, save_user_id)],
      TYPING_GOODREADS: [MessageHandler(Filters.text & ~Filters.command, save_goodreads_id)],
      SELECTING_ENGINE: [MessageHandler(Filters.regex('^KNN$'), warn_knn),
                         MessageHandler(Filters.regex('^SVD$'), warn_svd)]
            },
  fallbacks = [CommandHandler('start', start)]))

In [20]:
updater.start_polling()

In [26]:
updater.stop()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt

